# Digital Elevation Model
Download DEM and build attributes

- Digital Elevation Model from Amazon S3
- Data Engineering
- Data Plots

Sources:
- [DEM - Amazon S3](https://registry.opendata.aws/copernicus-dem/)
- [Documentation AWS](https://copernicus-dem-30m.s3.amazonaws.com/readme.html)

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)

## GeoLocation Data for Area of Interest

In [ ]:
import geopandas as gpd

geodf = gpd.read_file(f"./data/ThuenenGeoLocations/geolocations_aoi.geojson")
geodf = gpd.GeoDataFrame(geodf, geometry="geometry", crs="EPSG:4326")
geodf

#### Download Data

In [ ]:
def bounding_box(points):
    x_coordinates, y_coordinates = zip(*points)

    return [(min(x_coordinates), min(y_coordinates)), (max(x_coordinates), max(y_coordinates))]

items = dict()
bboxes = dict()

for i, row in geodf.iterrows():
    points = [point for point in row["geometry"].exterior.coords]
    bbox = bounding_box(points)
    
    bboxes[row["bez_wg_bu"]] = bbox
    items[row["bez_wg_bu"]] = []

    for x in range(int(bbox[0][0]), int(bbox[1][0]) + 1):
        for y in range(int(bbox[0][1]), int(bbox[1][1]) + 1):
            #print(f"Copernicus_DSM_COG_10_N{y}_00_E0{x:02d}_00_DEM/Copernicus_DSM_COG_10_N{y}_00_E0{x:02d}_00_DEM.tif")
            items[row["bez_wg_bu"]].append(f"Copernicus_DSM_COG_10_N{y}_00_E0{x:02d}_00_DEM/Copernicus_DSM_COG_10_N{y}_00_E0{x:02d}_00_DEM.tif")

    print(f"Getting DEM Data from Amazon S3 for coordinates to acquire {row['bez_wg_bu']}:")
    print(f"x:\t{int(bbox[0][0])} - {int(bbox[1][0])}")
    print(f"y:\t{int(bbox[0][1])} - {int(bbox[1][1])}")
    print()

items

#### Grafical: AOI Bounding Boxes

In [ ]:
import ipyleaflet
m = ipyleaflet.Map(scroll_wheel_zoom=False, center=(50, 8.5), zoom=6.35, height="400px")

# add controls
m.add_control(ipyleaflet.ScaleControl(position="bottomleft"))
m.add_control(ipyleaflet.FullScreenControl())

# blue = DEM Raster Data from S3
# red = AOI
# Orange = AOI Bounding Box

for area in bboxes:
    # define bbox for tile
    tile_bbox = ((bboxes[area][0][1], bboxes[area][0][0]), (bboxes[area][1][1], bboxes[area][1][0]))
    rectangle = ipyleaflet.Rectangle(bounds=tile_bbox, color="red", fill_opacity=0, weight=2)
    
    tile_bbox_big = ((int(bboxes[area][0][1]), int(bboxes[area][0][0])), (int(bboxes[area][1][1]) + 1, int(bboxes[area][1][0]) + 1))
    rectangle_big = ipyleaflet.Rectangle(bounds=tile_bbox_big, color="blue", fill_opacity=0, weight=2)
    
    # plot rectangle as new layer on map
    m.add_layer(rectangle)
    m.add_layer(rectangle_big)

tile_bbox = ((geodf.geometry.bounds["miny"].min(), geodf.geometry.bounds["minx"].min()), (geodf.geometry.bounds["maxy"].max(), geodf.geometry.bounds["maxx"].max()))
rectangle = ipyleaflet.Rectangle(bounds=tile_bbox, color="orange", fill_opacity=0, weight=2)
m.add_layer(rectangle)

m

## Load DEM Data

In [ ]:
import boto3
from botocore import UNSIGNED
from botocore.client import Config

In [ ]:
import re

BUCKET_NAME = "copernicus-dem-30m" # Digital Elevation Model - 30m resolution

# https://registry.opendata.aws/copernicus-dem/
s3 = boto3.resource("s3", config=Config(signature_version=UNSIGNED))
bucket = s3.Bucket(BUCKET_NAME)

s3_objects = dict()
s3_objects_list = list()
for area in items:
    s3_objects[area] = list()
    
for obj in bucket.objects.all():
    for area in items:
        if str(obj.key) in items[area]:
            s3_objects[area].append(obj)
            if obj not in s3_objects_list:
                s3_objects_list.append(obj)

s3_objects

In [ ]:
import rasterio as rio
from rasterio.session import AWSSession
from rasterio.windows import Window
import rioxarray
import os
os.environ['AWS_NO_SIGN_REQUEST'] = 'YES'

# create AWS session object
aws_session = AWSSession(boto3.Session())

data_arrays = list()

with rio.Env(aws_session):
    for obj in s3_objects_list:
        try:
            data_arrays.append(
                rioxarray.open_rasterio(f"s3://{obj.bucket_name}/{obj.key}")
            )
        except:
            pass
len(data_arrays)

## Data Preparation

In [ ]:
data_sets = [_da.to_dataset(name="elevation") for _da in data_arrays]

In [ ]:
import xarray as xr
xr.set_options(display_style="html")

ds = xr.merge(data_sets)

ds = ds.rio.write_crs(4326)
ds

In [ ]:
from pyproj import Proj, transform

def transform_epsg_32632_from_4326(x1):
    """EPSG 32632 from 4326"""
    inProj = Proj(init='epsg:4326')
    outProj = Proj(init='epsg:32632')
    x2, y2 = transform(inProj, outProj, x1, x1)
    return x2


### Change Resolution

In [ ]:
# Transform CRS 4326 to 32632 (Gauss-Krueger)
# EPSG 32632 unit = meters -> necessary to compare with hight above sea level
x_4326 = ds.x.values
y_4326 = ds.y.values

x_32632 = list()
y_32632 = list()

inProj = Proj(init="epsg:4326")
outProj = Proj(init="epsg:32632")
def transform_epsg_4326_32632(x1:float=None, y1:float=None):
    """using fixed value in bounds to calculate other"""
    x2, y2 = transform(inProj, outProj, x1, y1)
    return [x2, y2]

for _i, x in enumerate(ds.x.values):
    x2 = transform_epsg_4326_32632(x1=x, y1=ds.y.values[0])
    x_32632.append(x2[0])

for _i, y in enumerate(ds.y.values):
    y2 = transform_epsg_4326_32632(x1=ds.x.values[0], y1=y)
    y_32632.append(y2[1])

ds_32632 = ds.assign_coords(x=x_32632, y=y_32632)
ds_32632

In [ ]:
#ds_32632_clip = ds_32632.copy()
ds_32632_clip = ds_32632.copy()
ds_32632_clip = ds_32632_clip.sel(x=slice(geodf.to_crs(32632).geometry.bounds["minx"].min(), geodf.to_crs(32632).geometry.bounds["maxx"].max()), y=slice(geodf.to_crs(32632).geometry.bounds["miny"].min(), geodf.to_crs(32632).geometry.bounds["maxy"].max()))

In [ ]:
da = ds_32632_clip.to_array(dim="elevation")
da = da.drop_vars(["band", "elevation", "spatial_ref"])[0][0]

da = da.rio.write_crs(32632)

da

In [ ]:
# coarsen resoltion to approx 250m
SPACE_IN_BETWEEN_POINTS = 160
factor_x = int(int(ds_32632_clip.x.values[-1] - ds_32632_clip.x.values[0]) / int(SPACE_IN_BETWEEN_POINTS / 2))
factor_y = int(int(ds_32632_clip.y.values[-1] - ds_32632_clip.y.values[0]) / int(SPACE_IN_BETWEEN_POINTS / 2))

# copy of dataarray
da_coarsen = da.copy()

#da_coarsen = da_coarsen.reindex(x=x_new, y=y_new, method="nearest")
da_coarsen = da_coarsen.coarsen(x=int(len(da.x.values) / factor_x), y=int(len(da.y.values) / factor_y), boundary="trim").mean()  # +doctest: ELLIPSIS

da_coarsen

In [ ]:
# dataarray to dataset for file export
ds_coarsen = da_coarsen.to_dataset(name="elevation")

ds_coarsen.to_netcdf(path="./data/DEM/dem_coarsened.nc", engine="netcdf4")

In [ ]:
ds_coarsen

### Calculate Attributes

In [ ]:
ds_gradient = xr.Dataset(
    data_vars=dict(
        gradient_west_east=(["y", "x"], np.around(np.array(np.gradient(ds_coarsen.to_array(dim="elevation").values[0]))[0], decimals=4)),
        gradient_north_south=(["y", "x"], np.around(np.array(np.gradient(ds_coarsen.to_array(dim="elevation").values[0]))[1], decimals=4)),
        elevation=(["y", "x"], ds_coarsen.to_array(dim="elevation").values[0])
    ),
    coords=dict(
        x=(["x"], ds_coarsen.x.values),
        y=(["y"], ds_coarsen.y.values),
    ),
    attrs=dict(
        gradient_west_east="gradient from west to east (all y coordinates mapped to centered x coordinates)",
        gradient_north_south="gradient from north to west (all x coordinates mapped to centered y coordinates)",
        elevation="elevation above sea level (x, y)",
        crs="EPSG:32632"
    )
).rio.write_crs(32632)

In [ ]:
ds_gradient

#### Calculate Slope bestimmen

- West -> East:
    - positive: West
    - negative: East

- North -> South:
    - positive: North
    - negative: South

Direction:
```
       (N)
        1
 (W) 4  0  2 (E)
        3
       (S)
```

In [ ]:
bins = list()
_min = np.min(np.array([int(ds_gradient["gradient_west_east"].min()), int(ds_gradient["gradient_north_south"].min())]))
_max = np.max(np.array([int(ds_gradient["gradient_west_east"].max()), int(ds_gradient["gradient_north_south"].max())]))
_bound = np.max(np.array([abs(_max), abs(_max), abs(_min), abs(_min)]))

for i in range(0, _bound, 12):
    bins.append(i)
    
for i in range(0, (_bound*(-1)), -12):
    bins.append(i)
    
bins = np.unique(np.sort(np.array(bins)))

# cagegoriese values by bins
gradient_north_south_cat = (np.digitize(np.array(ds_gradient['gradient_north_south'].values), bins=bins) - len(bins)/2).astype(int)
gradient_west_east_cat = (np.digitize(np.array(ds_gradient['gradient_west_east'].values), bins=bins) - len(bins)/2).astype(int)

# set either north or south / west or east slope direction depending on gradient
gradient_north_south_direction = np.where((np.array(ds_gradient['gradient_north_south'].values) < 0.0), 3.0, 1.0)
gradient_west_east_direction = np.where((np.array(ds_gradient['gradient_west_east'].values) < 0.0), 4.0, 2.0)

# set None if there is no slope (category 0)
gradient_north_south_direction = np.where((gradient_north_south_cat == 0.0), 0.0, gradient_north_south_direction)
gradient_west_east_direction = np.where((gradient_west_east_cat == 0.0), 0.0, gradient_west_east_direction)

### Concatenate Altitude

In [ ]:
elevation_bins = list()
for i in range(int(np.nanmin(ds_gradient["elevation"].values)), int(np.nanmax(ds_gradient["elevation"].values)), 20):
    elevation_bins.append(i)

elevation_cat = (np.digitize(np.array(ds_gradient['elevation'].values), bins=elevation_bins))

In [ ]:
ds_gradient

In [ ]:
ds_gradient_parsed = xr.Dataset(
    data_vars=dict(
        gradient_west_east_cat=(["y", "x"], gradient_west_east_cat),
        gradient_north_south_cat=(["y", "x"], gradient_north_south_cat),
        gradient_west_east_direction=(["y", "x"], gradient_west_east_direction),
        gradient_north_south_direction=(["y", "x"], gradient_north_south_direction),
        elevation_cat=(["y", "x"], elevation_cat)
    ),
    coords=dict(
        x=(["x"], ds_coarsen.x.values),
        y=(["y"], ds_coarsen.y.values),
    ),
    attrs=dict(
        direction="shows slope diection if slope that has min gradient of (+-) approx 5% [n (1) e (2) s (3) w (4) None (0)]",
        gradient_west_east="gradient from west to east in steps of 12 (approx 5 percent)",
        gradient_north_south="gradient from north to west in steps of 12 (approx 5 percent)",
        elevation="elevation above sea level in steps of 10 (x, y) in steps of 10",
        crs="EPSG:32632"
    )
).rio.write_crs(32632)

ds_gradient_parsed.to_netcdf(path="./data/DEM/dem_parsed.nc", engine="netcdf4")

## Visuals
### Two-Dimensional

In [ ]:
%matplotlib inline
import ipyvolume as ipv
from matplotlib import cm

import hvplot.pandas
import geopandas as gpd
from bokeh.resources import INLINE
import bokeh.io
bokeh.io.output_notebook(INLINE)

In [ ]:
import geopandas as gpd
geodf = geodf.to_crs("EPSG:32632")

In [ ]:
geodf_filter = geodf.loc[geodf["bez_wg_bu"] == "Schwarzwald"]
geodf_filter

In [ ]:
ds_CUT = ds_gradient_parsed.rio.clip(geodf_filter.geometry, geodf_filter.crs)
ds_CUT = ds_CUT.where(ds_CUT != -9223372036854775808, drop=True) # 9223372036854775808 == NaN
ds_CUT = ds_CUT.coarsen(x=4, y=4, boundary="trim").mean()  # +doctest: ELLIPSIS
df_dem = ds_CUT.to_dataframe().reset_index().dropna()
df_dem

In [ ]:
df_dem.hvplot.points(x='x', y='y', width=550, hover_cols=["elevation_cat", "gradient_west_east_direction", "gradient_north_south_direction"], height=550, geo=True, color='elevation_cat', cmap="RdBu_r", alpha=.35, tiles='OSM', crs="EPSG:32632", title="Elevation Data for selected AOI (BlackForest)")

### Three-Dimensional

In [ ]:
x_coarsen = np.array([da_coarsen.x.values]*len(da_coarsen.y.values))
_y_coarsen = np.array([da_coarsen.y.values]*len(da_coarsen.x.values))

y_coarsen = np.transpose(_y_coarsen.copy())
z_coarsen = da_coarsen.values

In [ ]:
colormap = cm.coolwarm
znorm = z_coarsen - z_coarsen.min()
znorm /= znorm.ptp()
znorm.min(), znorm.max()
color = colormap(znorm)

ipv.figure()
ipv.plot_surface(x=x_coarsen, y=y_coarsen, z=z_coarsen, color=color[...,:3])
#ipv.plot_wireframe(x=x, y=y, z=arr.values[0], color="red")
ipv.xlim(x_coarsen.min(), x_coarsen.max())
ipv.ylim(y_coarsen.min(), y_coarsen.max())
ipv.zlim(z_coarsen.min(), z_coarsen.max())
ipv.show()